In [1]:
from scipy import io
import numpy as np
import os
import torch 
import dgl 
from dataprocess import dataloader, Arg, my_dataset

Using backend: pytorch


In [9]:
data = io.loadmat('E:\\DATABASE\\renzhi_BN\\EEG3\\dzq\\data.mat')

In [2]:
coh = io.loadmat('E:\\DATABASE\\renzhi_BN\\EEG3\\dzq\\adj.mat')
#名字和连接矩阵需要重组，按照节点数据的格式
# for key in coh:
#     print(key)


In [41]:
cohr = coh.items()

In [4]:
line1 = data['dzq']

In [17]:
line1[0,0].shape

(5, 1)

In [5]:
line2 = line1[0,0][1,0][2,0]

In [11]:
"""
    循环填充代码
    循环尺度最好选最大的且为2的倍数（幂指数）
"""
temp1 = np.c_[line2,line2,line2]
temp2 = temp1[:,0:512]
temp2.shape

(18, 512)

In [29]:
adj = np.ones((18,18))
edge_list = np.nonzero(adj)
g = dgl.convert.graph(edge_list)
#g.edata['f0'] = torch.tensor(line4.reshape(-1,1)).float()


In [42]:
g.ndata['h1'] = torch.tensor(line2).float() #节点数X特征长度 即可

In [43]:
g.ndata['h1'][0,0] 

tensor(4.6391)

In [69]:
from dgl.nn import GraphConv
conv = GraphConv(293, 4, norm='both', weight=True, bias=True)
res1 = conv(g, feat = g.ndata['h'], edge_weight = g.edata['f0'])
res2 = conv(g, feat = g.ndata['h'])

In [80]:
adj = np.ones((3,3))
e_l = adj.nonzero()
g = dgl.convert.graph(e_l)
g = dgl.add_self_loop(g)
feat1 = torch.ones(3, 1)
conv1 = GraphConv(1, 2, norm='both', weight=True, bias=True)
res1 = conv1(g, feat1)
conv2 = GraphConv(1, 2, norm='both', weight=True, bias=True)
feat2 = torch.ones(3,1)
res2 = conv2(g,feat1)
res1, res2

(tensor([[ 1.2509, -0.9039],
         [ 1.2509, -0.9039],
         [ 1.2509, -0.9039]], grad_fn=<AddBackward0>),
 tensor([[0.7069, 0.3094],
         [0.7069, 0.3094],
         [0.7069, 0.3094]], grad_fn=<AddBackward0>))

In [72]:
def collate(samples):
    #用map提取了samples的每个元素，在list里拼合再tensor，但应该是可以直接的
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    # 复杂的原因是特征只有一维，用迭代器才可以array
    batched_labels = torch.tensor(np.array([item.detach().numpy() for item in labels]))                                
    #batched_labels = torch.tensor(lables)
    return batched_graph, batched_labels

In [10]:
fdata = data['data'][0,0]
edata = data['data'][1,0]
fdata.shape , edata.shape

((5, 1), (5, 1))

In [11]:
f1 = fdata[0,0]
f2 = f1[0,0]
f1.shape ,f2.shape

((101, 1), (18, 244))

In [12]:
e1 = edata[0,0] # 试验次数
e2 = e1[0,0] # 频率
e3 = e2[0,0]
e1.shape,e2.shape,e3.shape

((101, 1), (33, 1), (18, 18))

In [19]:
a = torch.tensor([[0.,1.,0.],[0.,0.,1.]])
b = torch.tensor([[1],[2]])

In [30]:
a_n = torch.nn.functional.log_softmax(a,dim = 1)
b_n = torch.zeros_like(a_n)
b_n = b_n.scatter(dim=1,index = b,value = 1)
a_n,b_n

(tensor([[-1.5514, -0.5514, -1.5514],
         [-1.5514, -1.5514, -0.5514]]),
 tensor([[0., 1., 0.],
         [0., 0., 1.]]))

In [27]:
cha1 = torch.sqrt(torch.mul(a_n,b_n))
cha = cha1.mean()*cha1.shape[0]*cha1.shape[1]
cha1,cha

(tensor([[0.0000, 0.8712, 0.0000],
         [0.0000, 0.0000, 0.8712]]),
 tensor(1.7424))

In [33]:
cha1.max(1)[1] == b

tensor([[ True, False],
        [False,  True]])

In [25]:
   res = 2-cha
   res

tensor(0.2576)

In [28]:
acc = (1/torch.sqrt(torch.tensor(2))*torch.sqrt(res))
acc

tensor(0.3589)

In [2]:
orp = 0
drp = 128
orp,drp = orp+128,drp+128
orp,drp

(128, 256)

In [24]:
pred = torch.randn((4, 5))
print(pred)
values, indices = pred.topk(1, dim=1, largest=True, sorted=True)
print(values, indices)

tensor([[ 1.0655, -0.7096,  0.6405, -2.0533,  1.9299],
        [-0.9971,  0.4101, -1.4562, -2.0961,  1.0172],
        [ 0.2582, -1.1407,  0.0614, -1.3147,  0.3592],
        [ 0.3762,  1.0997,  1.1899,  1.2995, -0.2576]])
tensor([[1.9299],
        [1.0172],
        [0.3592],
        [1.2995]]) tensor([[4],
        [4],
        [4],
        [3]])


In [31]:
lables = torch.tensor([[1],[2],[4],[3]])

In [33]:
acc = (indices == lables).float().mean()
acc

tensor(0.5000)

In [15]:
list1 = eac.nonzero()
list1

tensor([[0, 0],
        [0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [2, 0],
        [2, 2],
        [2, 3],
        [2, 4],
        [3, 1]])

In [23]:
# list1 = list1.tolist()
len(list1)

10

In [2]:
# 2021年4月27日21:06:30
arg = Arg()
dataset = my_dataset(raw_dir=arg.raw_dir,save_dir=arg.save_dir)

In [40]:
indict = []
index = 0
ran = int(len(dataset)/3)
for i in range(ran):
    chose = np.random.randint(1,4)
    indict.append(index+chose)
    index = 3+index
indict[0:10]

[1, 5, 9, 10, 14, 17, 20, 23, 27, 28]

In [ ]:
indict = range(len(dataset))
split_n = int(len(dataset)/dataset.num_labels*0.8)
ind2 = []

In [ ]:
indict = [range(300),range(375,675),range(750,1125),range()]

In [43]:
ori = 0
tr_indict = []
te_indict = []
for i in range(5):
    tr_indict.append(range(ori,ori+300))
    te_indict.append(range(ori+300,ori+375))
    ori+=375
tr_indict , te_indict 

([range(0, 300),
  range(375, 675),
  range(750, 1050),
  range(1125, 1425),
  range(1500, 1800)],
 [range(300, 375),
  range(675, 750),
  range(1050, 1125),
  range(1425, 1500),
  range(1800, 1875)])

In [44]:
train_simpler = torch.utils.data.sampler.SubsetRandomSampler(tr_indict)